<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/portfolios/optimal_sb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from cvxopt import matrix as Matrix
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False

def optimal_s(rs, rb, means, cov, raver):
    n = len(means)
    Q = np.zeros((n + 2, n + 2))
    Q[2:, 2:] = raver * cov
    Q = Matrix(Q, tc="d")                               # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc="d")                  # coefficients of objective function
    g = np.array([1, -1] + n * [1])
    a = np.zeros((2, n + 2))
    a[0, 0] = a[1, 1] = -1
    G = Matrix(np.vstack([a, g]))                       # coefficients of constraints
    h = np.array(2 * [0] + [1])
    h = Matrix(h, (len(h), 1), tc="d")                  # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None

def optimal_ns(rs, rb, means, cov, raver) :
    n = len(means)
    Q = np.zeros((n+2, n+2))
    Q[2:,2:] = raver*cov
    Q = Matrix(Q, tc='d')                                             # quadratic form that penalizes variance
    p = np.array([-rs, rb] + list(-np.array(means)))
    p = Matrix(p, (len(p), 1), tc='d')                                # coefficients of objective function
    g = np.array([1, -1] + n*[1])
    G = Matrix(np.vstack([-np.identity(n+2), g]), tc='d')             # coefficients of constraints
    h = np.array((n+2)*[0] + [1])
    h = Matrix(h, (len(h), 1), tc='d')                                # right-hand side of constraints
    sol = Solver(Q, p, G, h)
    return sol['x'] if sol['status']=='optimal' else None


In [ ]:
import ssl
import pandas as pd

# Get returns data
ssl._create_default_https_context = ssl._create_unverified_context
file = "http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls"
sheet = "Returns by year"
df = pd.read_excel(file, sheet_name=sheet, skiprows=16, header=[0, 1])
col1 = ("Unnamed: 0_level_0", "Year")
indx = df[df[col1].isna()].index[0]
df = df.iloc[:indx].set_index(col1)
df.index.name = "Year"

df = df["Annual Real Returns on"].copy()
df = df[df.columns[1:5]]
df.columns = ["S&P 500", "TBills", "Treasuries", "Corporates"]
df.dtype = "float"
df.index = [int(x) for x in df.index]
df.index.name = "Year"
df = df.drop(columns=["TBills"])

# displaying the first 5 rows of df
df.head()

/Users/kailiao/PycharmProjects/appcode_draft/venv/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,S&P 500,Treasuries,Corporates
Year,,,
1928,0.454932,0.020148,0.044268
1929,-0.088311,0.035980,0.024189
1930,-0.200079,0.116835,0.074090
1931,-0.380674,0.074522,-0.070178
1932,0.018184,0.212472,0.377411
1933,0.488460,0.010836,0.121111
1934,-0.026634,0.063520,0.170431
1935,0.424871,0.014439,0.100235
1936,0.300585,0.035176,0.097926


In [ ]:
# solutions
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
sol = optimal_ns(rs, rb, means, cov, raver)


if sol:
    sol = list(sol)
    print('Without Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    print(f'optimal investment in asset S&P 500 is {sol[2]:.1%}')
    print(f'optimal investment in asset Treasuries is {sol[3]:.1%}')
    print(f'optimal investment in asset Corporates is {sol[4]:.1%}')

Without Short Sales:
optimal amount to save is 0.0%
optimal amount to borrow is 0.0%
optimal investment in asset S&P 500 is 73.4%
optimal investment in asset Treasuries is 0.0%
optimal investment in asset Corporates is 26.6%


In [ ]:
# solutions
means = df.mean().to_numpy()
cov = df.cov().to_numpy()
rs = 0.02                             # example savings rate
rb = 0.06                             # example borrowing rate
raver = 2                             # example risk aversion
sol = optimal_s(rs, rb, means, cov, raver)


if sol:
    sol = list(sol)
    print('With Short Sales:')
    print(f'optimal amount to save is {sol[0]:.1%}')
    print(f'optimal amount to borrow is {sol[1]:.1%}')
    print(f'optimal investment in asset S&P 500 is {sol[2]:.1%}')
    print(f'optimal investment in asset Treasuries is {sol[3]:.1%}')
    print(f'optimal investment in asset Corporates is {sol[4]:.1%}')

With Short Sales:
optimal amount to save is 0.0%
optimal amount to borrow is -0.0%
optimal investment in asset S&P 500 is 58.4%
optimal investment in asset Treasuries is -122.7%
optimal investment in asset Corporates is 164.3%
